# Session 2: [Fast-Track] AI 분석 전문가 구축 및 배포 시작

## 🎯 세션 목표
본 세션은 **50분** 안에 고성능 **AI 분석 전문가 (대출 심사 모델)**를 구축하고 배포하는 모든 과정을 압축 실행합니다.
-   **XGBoost** 라이브러리를 설치하고 사용합니다.
-   모델 학습 후 GCS에 저장하고 Vertex AI Endpoint 배포를 **시작**합니다.
-   배포가 완료되길 기다리지 않고 바로 다음 세션으로 넘어갑니다.

### ⚠️ 중요 환경 설정
Vertex AI 컨테이너와의 호환성을 위해 **scikit-learn 버전을 자동으로 1.5.0으로 맞춥니다.** 코드를 순서대로 실행해 주세요.

In [ ]:
# [코드 셀 1] 환경 설정 및 라이브러리 설치

import sys
import os
import pandas as pd
import joblib
import sklearn
from google.cloud import aiplatform
from google.cloud import storage
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier # 고성능 모델인 XGBoost 임포트
from sklearn.metrics import accuracy_score

# --- (1) XGBoost 라이브러리 설치 ---
# Vertex AI Workbench에는 기본으로 설치되어 있지 않을 수 있으므로, 명시적으로 설치합니다.
print("XGBoost 라이브러리 설치를 시작합니다...")
!pip install xgboost --quiet
print("XGBoost 설치 완료.")

# --- (2) Scikit-learn 버전 검사 및 조정 ---
REQUIRED_SKLEARN_VERSION = "1.5.0"

if sklearn.__version__ != REQUIRED_SKLEARN_VERSION:
    print(f"경고: 현재 scikit-learn 버전({sklearn.__version__})이 요구 버전({REQUIRED_SKLEARN_VERSION})과 다릅니다.")
    print(f"자동으로 {REQUIRED_SKLEARN_VERSION} 버전으로 다운그레이드/설치를 진행합니다.")
    !pip install scikit-learn=={REQUIRED_SKLEARN_VERSION} --quiet
    
    # 설치 후 커널 재시작 안내 (필수)
    print(f"\n====================================================================================")
    print(f"▶ scikit-learn {REQUIRED_SKLEARN_VERSION} 설치 완료! ")
    print(f"▶ **JupyterLab 'Kernel' -> 'Restart Kernel...'을 클릭하여 커널을 재시작**한 후,")
    print(f"▶ 이 셀부터 다시 실행해 주세요.")
    print(f"====================================================================================")
else:
    print(f"현재 scikit-learn 버전: {sklearn.__version__}. 환경 설정 완료.")

## 📝 데이터 로드 및 전처리 함수 정의

* ML 전문가 모델은 숫자로 된 데이터만 이해
* 이 단계에서는 복잡한 전처리 과정을 하나의 함수로 정의
* 이 함수는 원본 데이터를 받아 **결측치를 처리하고, 텍스트(범주형) 데이터를 숫자로 변환(One-Hot Encoding)**

In [ ]:
# [코드 셀 2] 데이터 전처리 함수 정의

def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    ML 모델 학습을 위해 데이터를 전처리하는 함수를 정의합니다.
    """
    # 1. 불필요한 'Loan_ID' 컬럼 제거
    df = df.drop('Loan_ID', axis=1, errors='ignore')
    
    # 2. 결측치(NaN) 처리: 최빈값과 중앙값으로 대체
    cat_cols = ['Gender', 'Married', 'Dependents', 'Self_Employed', 'Credit_History']
    for col in cat_cols:
        df[col] = df[col].fillna(df[col].mode()[0])
    num_cols = ['LoanAmount', 'Loan_Amount_Term']
    for col in num_cols:
        df[col] = df[col].fillna(df[col].median())
        
    # 3. 범주형 변수 인코딩 (One-Hot Encoding)
    df_processed = pd.get_dummies(df, drop_first=True, columns=[
        'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area'
    ])
    
    return df_processed

print("데이터 전처리 함수 (preprocess_data) 정의 완료.")

In [ ]:
# [코드 셀 3] 데이터 로드, 학습 및 아티팩트 저장

# 🚨 사용자 설정값 (반드시 수정)
BUCKET_NAME = "여러분의-GCS-버킷-이름" # 예: "gcp-project-12345-loan-data"
TRAIN_FILE = "train_u6lujuX_CVtuZ9i.csv" # Kaggle 대출 심사 원본 파일명

# GCS 경로 설정
GCS_PATH = f"gs://{BUCKET_NAME}/{TRAIN_FILE}"

# GCS에서 데이터 로드 (Step 1)
try:
    df = pd.read_csv(GCS_PATH)
    print(f"1. GCS에서 데이터 로드 완료. 총 {df.shape[0]} 건.")
except Exception as e:
    print(f"🚨 오류: GCS 로드 실패! 버킷 이름 또는 파일 존재 여부를 확인하세요. 오류: {e}")
    sys.exit() # 오류 시 스크립트 중단

# 데이터 전처리 실행 (Step 2)
df_processed = preprocess_data(df)
df_processed['Loan_Status'] = df_processed['Loan_Status_Y'].astype(int)

# 피처(X)와 타겟(y) 분리 및 학습/검증 분리
X = df_processed.drop('Loan_Status_Y', axis=1)
y = df_processed['Loan_Status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("2. 데이터 전처리 및 학습/검증 데이터 분리 완료.")


# XGBoost 모델 학습 (Step 3)
model = XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"3. XGBoost 모델 학습 완료. 검증 정확도: {accuracy_score(y_test, y_pred):.4f}")


# 모델 아티팩트 로컬 저장 (Step 4)
MODEL_DIR = 'models'
os.makedirs(MODEL_DIR, exist_ok=True)
joblib.dump(model, os.path.join(MODEL_DIR, 'loan_approval_model.pkl'))
joblib.dump(X.columns.tolist(), os.path.join(MODEL_DIR, 'model_features.pkl'))
print("4. 모델 아티팩트 로컬 저장 완료.")

## 📝 Vertex AI 등록 및 배포 함수 정의

모델 아티팩트를 GCS에 업로드하고 Model Registry에 등록한 뒤, Endpoint에 배포하는 모든 과정을 함수로 정의합니다.
이 함수는 복잡한 단계를 자동으로 처리합니다.

In [ ]:
# [코드 셀 4] Vertex AI 자원 등록 및 배포 함수 정의

def register_and_deploy_model(project_id, region, bucket_name, endpoint_name, model_name):
    
    # 1. GCS 업로드 (Step 5)
    MODEL_GCS_PATH = f"gs://{bucket_name}/loan_model/v1/"
    ARTIFACT_GCS_PATH = f"gs://{bucket_name}/loan_model/artifacts/"

    # 모델 파일 및 피처 리스트를 GCS에 분리하여 업로드 (UI 등록 오류 방지)
    !gsutil cp models/loan_approval_model.pkl {MODEL_GCS_PATH}
    !gsutil cp models/model_features.pkl {ARTIFACT_GCS_PATH}
    print("1. GCS 업로드 완료.")

    # 2. Model Registry 등록 (Step 6)
    aiplatform.init(project=project_id, location=region)
    
    # Model Registry에 등록
    uploaded_model = aiplatform.Model.upload(
        display_name=model_name,
        artifact_uri=MODEL_GCS_PATH, # GCS 폴더 경로
        serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/xgboost-gpu.1-5:latest", # XGBoost 컨테이너 (버전은 Scikit-learn 컨테이너와 호환되는 최신 버전 사용)
        serving_container_environment_variables={"MODEL_FILENAME": "loan_approval_model.pkl"},
        sync=True # 동기식 등록
    )
    print(f"2. Model Registry 등록 완료. 모델 ID: {uploaded_model.name}")
    
    # 3. Endpoint 배포 시작 (Step 7)
    # Endpoint 리소스 정의
    endpoint = aiplatform.Endpoint.create(display_name=endpoint_name, project=project_id, location=region)
    
    # 모델 배포 시작 (비동기 처리)
    endpoint.deploy(
        model=uploaded_model,
        deployed_model_display_name=f"{model_name}-deployed",
        machine_type="n1-standard-2", # 배포 VM 사양
        min_replica_count=1,
        max_replica_count=1,
        sync=False # 비동기 배포 (함수를 즉시 종료하고 백그라운드에서 실행)
    )
    print("3. Endpoint 배포 요청 시작. 배포 완료까지 10~15분 소요됩니다.")
    
    return endpoint

print("Vertex AI 자원 등록 및 배포 함수 정의 완료.")

In [ ]:
# [코드 셀 5] 최종 등록 및 배포 실행

# 🚨 사용자 설정값 (코드 셀 1의 값과 일치해야 함)
PROJECT_ID = "여러분의-GCP-PROJECT-ID"
BUCKET_NAME = "여러분의-GCS-버킷-이름"
REGION = "asia-northeast3"

# 배포할 Endpoint와 Model 이름 정의
ENDPOINT_NAME = "loan-approval-service"
MODEL_NAME = "loan-analysis-expert"

# 함수 실행 (전체 워크플로우 시작)
deployed_endpoint = register_and_deploy_model(
    project_id=PROJECT_ID,
    region=REGION,
    bucket_name=BUCKET_NAME,
    endpoint_name=ENDPOINT_NAME,
    model_name=MODEL_NAME
)

print("\n====================================================================================")
print("▶▶▶ Session 2 완료. Endpoint 배포가 백그라운드에서 진행 중입니다. ◀◀◀")
print("▶ 다음 세션(Session 3)에서 이 Endpoint를 사용하게 됩니다.")
print("====================================================================================")